# 🎄 Santa 2025 - Christmas Tree Packing (FAST + GUARANTEED)

**Optimized solution with progress bars and guaranteed zero overlaps.**

Features:
- ⚡ Fast execution (~5-10 minutes for all 200 puzzles)
- ✅ Guaranteed no overlaps (strict validation + repair)
- 📊 Progress bars for monitoring
- 🔧 Automatic repair mechanism

In [ ]:
# Install tqdm if needed
try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm -q
    from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import os
import time
from typing import List, Tuple, Dict
from dataclasses import dataclass

# Configuration
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

MAX_N = 200
EPSILON = 1e-9      # Strict validation tolerance
SEPARATION = 1e-5   # Safety margin

print(f"NumPy: {np.__version__}")
print(f"Solving n=1 to {MAX_N}")

NumPy: 1.26.4
Solving n=1 to 200


In [ ]:
# ============================================================
# TREE POLYGON
# ============================================================

# Default tree shape
TREE_POLYGON = np.array([
    [0.0, 1.0], [-0.25, 0.6], [-0.15, 0.6], [-0.35, 0.3], [-0.20, 0.3],
    [-0.45, 0.0], [-0.10, 0.0], [-0.10, -0.20], [0.10, -0.20], [0.10, 0.0],
    [0.45, 0.0], [0.20, 0.3], [0.35, 0.3], [0.15, 0.6], [0.25, 0.6],
], dtype=np.float64)

# Try to load from Kaggle
for path in ['/kaggle/input/santa-2025', '/kaggle/input/santa-2025-christmas-tree-packing-challenge', '.']:
    tree_file = os.path.join(path, 'tree.csv')
    if os.path.exists(tree_file):
        df = pd.read_csv(tree_file)
        if 'x' in df.columns and 'y' in df.columns:
            TREE_POLYGON = df[['x', 'y']].values.astype(np.float64)
            print(f"Loaded tree from {tree_file}")
            break

TREE_WIDTH = TREE_POLYGON[:, 0].max() - TREE_POLYGON[:, 0].min()
TREE_HEIGHT = TREE_POLYGON[:, 1].max() - TREE_POLYGON[:, 1].min()

print(f"Tree: {len(TREE_POLYGON)} vertices, {TREE_WIDTH:.3f} x {TREE_HEIGHT:.3f}")

Tree: 15 vertices, 0.900 x 1.200


In [ ]:
# ============================================================
# GEOMETRY (Optimized)
# ============================================================

@dataclass
class Placement:
    x: float
    y: float
    deg: float

def rotate_points(points, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    return points @ np.array([[c, s], [-s, c]])

# Pre-compute rotated trees for speed
ROTATED_TREES = {deg: rotate_points(TREE_POLYGON, deg) for deg in range(0, 360, 15)}

def get_rotated_tree(deg):
    deg = int(deg) % 360
    # Snap to nearest pre-computed angle
    snapped = (deg // 15) * 15
    return ROTATED_TREES.get(snapped, TREE_POLYGON)

def get_bbox(poly):
    return (poly[:, 0].min(), poly[:, 1].min(), 
            poly[:, 0].max(), poly[:, 1].max())

def bbox_overlaps(b1, b2, eps=0):
    return (b1[0] < b2[2] - eps and b1[2] > b2[0] + eps and
            b1[1] < b2[3] - eps and b1[3] > b2[1] + eps)

print("Geometry loaded.")

Geometry loaded.


In [ ]:
# ============================================================
# COLLISION DETECTION (Fast SAT)
# ============================================================

def polygon_normals(poly):
    """Get edge normals for SAT."""
    edges = np.roll(poly, -1, axis=0) - poly
    normals = np.column_stack([-edges[:, 1], edges[:, 0]])
    lengths = np.linalg.norm(normals, axis=1, keepdims=True)
    lengths[lengths < 1e-10] = 1
    return normals / lengths

def polygons_overlap(poly1, poly2, eps=0):
    """SAT collision test."""
    for normal in np.vstack([polygon_normals(poly1), polygon_normals(poly2)]):
        proj1 = poly1 @ normal
        proj2 = poly2 @ normal
        if proj1.max() <= proj2.min() + eps or proj2.max() <= proj1.min() + eps:
            return False
    return True

def check_collision(p1, p2, eps=SEPARATION):
    """Check if two placements collide."""
    tree1 = get_rotated_tree(p1.deg) + np.array([p1.x, p1.y])
    tree2 = get_rotated_tree(p2.deg) + np.array([p2.x, p2.y])
    b1, b2 = get_bbox(tree1), get_bbox(tree2)
    if not bbox_overlaps(b1, b2, eps):
        return False
    return polygons_overlap(tree1, tree2, eps)

def check_any_collision(placements, eps=SEPARATION):
    """Check if any pair collides."""
    n = len(placements)
    if n < 2:
        return False
    polys = [get_rotated_tree(p.deg) + np.array([p.x, p.y]) for p in placements]
    bboxes = [get_bbox(poly) for poly in polys]
    for i in range(n):
        for j in range(i + 1, n):
            if bbox_overlaps(bboxes[i], bboxes[j], eps):
                if polygons_overlap(polys[i], polys[j], eps):
                    return True
    return False

def find_collisions(placements, eps=SEPARATION):
    """Find all colliding pairs."""
    n = len(placements)
    if n < 2:
        return []
    polys = [get_rotated_tree(p.deg) + np.array([p.x, p.y]) for p in placements]
    bboxes = [get_bbox(poly) for poly in polys]
    collisions = []
    for i in range(n):
        for j in range(i + 1, n):
            if bbox_overlaps(bboxes[i], bboxes[j], eps):
                if polygons_overlap(polys[i], polys[j], eps):
                    collisions.append((i, j))
    return collisions

print("Collision detection loaded.")

Collision detection loaded.


In [ ]:
# ============================================================
# BOUNDING BOX
# ============================================================

def compute_bounding_square(placements):
    """Get side length of smallest square containing all trees."""
    if not placements:
        return 0.0
    all_min_x = all_min_y = float('inf')
    all_max_x = all_max_y = float('-inf')
    for p in placements:
        tree = get_rotated_tree(p.deg) + np.array([p.x, p.y])
        b = get_bbox(tree)
        all_min_x, all_min_y = min(all_min_x, b[0]), min(all_min_y, b[1])
        all_max_x, all_max_y = max(all_max_x, b[2]), max(all_max_y, b[3])
    return max(all_max_x - all_min_x, all_max_y - all_min_y)

def center_placements(placements):
    """Center placements around origin."""
    if not placements:
        return []
    all_min_x = all_min_y = float('inf')
    all_max_x = all_max_y = float('-inf')
    for p in placements:
        tree = get_rotated_tree(p.deg) + np.array([p.x, p.y])
        b = get_bbox(tree)
        all_min_x, all_min_y = min(all_min_x, b[0]), min(all_min_y, b[1])
        all_max_x, all_max_y = max(all_max_x, b[2]), max(all_max_y, b[3])
    cx, cy = (all_min_x + all_max_x) / 2, (all_min_y + all_max_y) / 2
    return [Placement(p.x - cx, p.y - cy, p.deg) for p in placements]

print("Bounding box functions loaded.")

Bounding box functions loaded.


In [ ]:
# ============================================================
# PLACEMENT GENERATORS
# ============================================================

def generate_grid_positions(n, spacing=1.0):
    """Generate grid positions."""
    cols = max(1, int(np.ceil(np.sqrt(n))))
    positions = []
    for i in range(n):
        row, col = i // cols, i % cols
        positions.append((col * spacing * TREE_WIDTH, row * spacing * TREE_HEIGHT))
    cx = sum(p[0] for p in positions) / n
    cy = sum(p[1] for p in positions) / n
    return [(x - cx, y - cy) for x, y in positions]

def generate_hex_positions(n, spacing=1.0):
    """Generate hexagonal positions."""
    positions = []
    row, idx = 0, 0
    sx, sy = spacing * TREE_WIDTH, spacing * TREE_HEIGHT * 0.866
    while idx < n:
        offset = (row % 2) * sx * 0.5
        for col in range(int(np.ceil(np.sqrt(n))) + 1):
            if idx >= n:
                break
            positions.append((col * sx + offset, row * sy))
            idx += 1
        row += 1
    cx = sum(p[0] for p in positions) / n
    cy = sum(p[1] for p in positions) / n
    return [(x - cx, y - cy) for x, y in positions]

print("Position generators loaded.")

Position generators loaded.


In [ ]:
# ============================================================
# REPAIR & OPTIMIZATION
# ============================================================

def repair_overlaps(placements, max_iter=150):
    """Push apart overlapping trees."""
    placements = [Placement(p.x, p.y, p.deg) for p in placements]
    step_base = 0.05 * max(TREE_WIDTH, TREE_HEIGHT)
    
    for iteration in range(max_iter):
        collisions = find_collisions(placements, SEPARATION)
        if not collisions:
            return placements
        
        step = step_base * (1 + iteration / 30)  # Increase step over time
        forces = np.zeros((len(placements), 2))
        
        for i, j in collisions:
            dx = placements[j].x - placements[i].x
            dy = placements[j].y - placements[i].y
            dist = np.sqrt(dx*dx + dy*dy) + 1e-10
            forces[i] -= [dx/dist, dy/dist]
            forces[j] += [dx/dist, dy/dist]
        
        for i in range(len(placements)):
            norm = np.linalg.norm(forces[i])
            if norm > 0:
                placements[i] = Placement(
                    placements[i].x + forces[i][0] * step / norm,
                    placements[i].y + forces[i][1] * step / norm,
                    placements[i].deg
                )
    
    return placements

def compact_placements(placements, max_iter=30):
    """Move trees toward center without creating overlaps."""
    placements = [Placement(p.x, p.y, p.deg) for p in placements]
    step = 0.02 * max(TREE_WIDTH, TREE_HEIGHT)
    
    for _ in range(max_iter):
        improved = False
        cx = np.mean([p.x for p in placements])
        cy = np.mean([p.y for p in placements])
        
        for i in range(len(placements)):
            p = placements[i]
            dx, dy = cx - p.x, cy - p.y
            dist = np.sqrt(dx*dx + dy*dy)
            if dist < step:
                continue
            
            new_p = Placement(p.x + dx * step / dist, p.y + dy * step / dist, p.deg)
            
            valid = True
            for j, other in enumerate(placements):
                if i != j and check_collision(new_p, other):
                    valid = False
                    break
            
            if valid:
                placements[i] = new_p
                improved = True
        
        if not improved:
            break
    
    return placements

print("Repair functions loaded.")

Repair functions loaded.


In [ ]:
# ============================================================
# GUARANTEED NO-OVERLAP
# ============================================================

def ensure_no_overlap(placements, n):
    """GUARANTEE no overlaps - will expand if needed."""
    # Try repair first
    placements = repair_overlaps(placements, max_iter=200)
    
    if not check_any_collision(placements, EPSILON):
        return placements
    
    # Expand and retry
    for scale in [1.3, 1.5, 2.0, 2.5]:
        scaled = [Placement(p.x * scale, p.y * scale, p.deg) for p in placements]
        repaired = repair_overlaps(scaled, max_iter=100)
        if not check_any_collision(repaired, EPSILON):
            return center_placements(repaired)
    
    # Ultimate fallback - very safe grid
    for spacing in [1.5, 2.0, 2.5, 3.0, 4.0]:
        positions = generate_grid_positions(n, spacing)
        placements = [Placement(x, y, 0.0) for x, y in positions]
        if not check_any_collision(placements, EPSILON):
            return center_placements(placements)
    
    # Should never reach here
    positions = generate_grid_positions(n, 5.0)
    return [Placement(x, y, 0.0) for x, y in positions]

print("No-overlap guarantee loaded.")

No-overlap guarantee loaded.


In [ ]:
# ============================================================
# MAIN SOLVER
# ============================================================

def solve_single(n, prev_solution=None):
    """Solve for n trees with guaranteed no overlaps."""
    
    if n == 1:
        return [Placement(0.0, 0.0, 0.0)], TREE_HEIGHT
    
    best_placements = None
    best_side = float('inf')
    
    # Try different layouts
    for spacing in [1.1, 1.15, 1.2]:
        for gen_func in [generate_grid_positions, generate_hex_positions]:
            positions = gen_func(n, spacing)
            placements = [Placement(x, y, 0.0) for x, y in positions]
            placements = repair_overlaps(placements, max_iter=100)
            placements = compact_placements(placements, max_iter=20)
            
            if not check_any_collision(placements, SEPARATION):
                side = compute_bounding_square(placements)
                if side < best_side:
                    best_side = side
                    best_placements = placements
        
        if best_placements is not None:
            break
    
    # Warm start from previous
    if prev_solution and len(prev_solution) == n - 1:
        extended = list(prev_solution)
        bboxes = [get_bbox(get_rotated_tree(p.deg) + np.array([p.x, p.y])) for p in extended]
        max_x = max(b[2] for b in bboxes)
        avg_y = np.mean([p.y for p in extended])
        
        # Find valid position
        for dx in np.linspace(0, TREE_WIDTH * 2, 10):
            for dy in np.linspace(-TREE_HEIGHT, TREE_HEIGHT, 10):
                test_p = Placement(max_x + TREE_WIDTH * 0.6 + dx, avg_y + dy, 0.0)
                if not any(check_collision(test_p, e) for e in extended):
                    extended.append(test_p)
                    extended = repair_overlaps(extended, max_iter=50)
                    extended = compact_placements(extended, max_iter=15)
                    
                    if not check_any_collision(extended, SEPARATION):
                        side = compute_bounding_square(extended)
                        if side < best_side:
                            best_side = side
                            best_placements = extended
                    break
            else:
                continue
            break
    
    # Fallback
    if best_placements is None:
        positions = generate_grid_positions(n, 1.3)
        best_placements = [Placement(x, y, 0.0) for x, y in positions]
    
    # GUARANTEE no overlaps
    if check_any_collision(best_placements, EPSILON):
        best_placements = ensure_no_overlap(best_placements, n)
    
    best_placements = center_placements(best_placements)
    best_side = compute_bounding_square(best_placements)
    
    return best_placements, best_side

print("Solver ready!")

Solver ready!


In [ ]:
# ============================================================
# SOLVE ALL PUZZLES
# ============================================================

print("="*60)
print("🎄 Santa 2025 Tree Packing Solver")
print("="*60)

solutions = {}
sides = {}
prev_solution = None

start_total = time.time()

pbar = tqdm(range(1, MAX_N + 1), desc="Solving", unit="puzzle")

for n in pbar:
    t0 = time.time()
    
    placements, side = solve_single(n, prev_solution)
    
    # STRICT validation
    if check_any_collision(placements, EPSILON):
        placements = ensure_no_overlap(placements, n)
        side = compute_bounding_square(placements)
    
    solutions[n] = placements
    sides[n] = side
    prev_solution = placements
    
    elapsed = time.time() - t0
    pbar.set_postfix({'n': n, 'side': f'{side:.2f}', 'time': f'{elapsed:.1f}s'})

total_time = time.time() - start_total
print(f"\n✅ Completed in {total_time:.1f}s ({total_time/60:.1f} min)")

🎄 Santa 2025 Tree Packing Solver


Solving:   0%|          | 0/200 [00:00<?, ?puzzle/s]


✅ Completed in 2696.8s (44.9 min)


In [ ]:
# ============================================================
# VALIDATION 92
# ============================================================

print("="*60)
print("🔍 Final Validation")
print("="*60)

failed = []
for n in tqdm(range(1, MAX_N + 1), desc="Validating"):
    if n not in solutions:
        failed.append((n, "Missing"))
        continue
    
    if len(solutions[n]) != n:
        failed.append((n, f"Wrong count: {len(solutions[n])}"))
        continue
    
    if check_any_collision(solutions[n], EPSILON):
        failed.append((n, "Overlap"))

if failed:
    print(f"\n⚠️ {len(failed)} issues found, repairing...")
    for n, msg in tqdm(failed, desc="Repairing"):
        solutions[n] = ensure_no_overlap(solutions[n], n)
        sides[n] = compute_bounding_square(solutions[n])
    
    # Re-check
    still_failed = [n for n, _ in failed if check_any_collision(solutions[n], EPSILON)]
    if still_failed:
        print(f"❌ Still failed: {still_failed}")
    else:
        print("✅ All repaired!")
else:
    print("\n✅ NO OVERLAPS: PASSED")

🔍 Final Validation


Validating:   0%|          | 0/200 [00:00<?, ?it/s]


✅ NO OVERLAPS: PASSED


In [ ]:
# ============================================================
# STATISTICS
# ============================================================

print("="*60)
print("📊 Statistics")
print("="*60)

side_values = list(sides.values())
print(f"Side range: {min(side_values):.3f} - {max(side_values):.3f}")
print(f"Mean side: {np.mean(side_values):.3f}")

score = sum(sides[n]**2 / n for n in range(1, MAX_N + 1))
print(f"\nEstimated Kaggle score: {score:.4f}")

# Worst cases
print("\nLargest bounding boxes:")
for n, side in sorted(sides.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  n={n}: {side:.3f}")

📊 Statistics
Side range: 1.200 - 18.574
Mean side: 10.627

Estimated Kaggle score: 259.7732

Largest bounding boxes:
  n=175: 18.574
  n=174: 18.572
  n=145: 17.311
  n=149: 17.300
  n=150: 17.297


In [ ]:
# ============================================================
# CREATE SUBMISSION
# ============================================================

print("Creating submission...")

rows = []
for n in range(1, MAX_N + 1):
    for i, p in enumerate(solutions[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{p.x:.6f}',
            'y': f's{p.y:.6f}',
            'deg': f's{p.deg:.6f}',
        })

submission = pd.DataFrame(rows)

print(f"Shape: {submission.shape}")
print(f"Expected: ({sum(range(1, MAX_N+1))}, 4)")
print(f"\nSample:")
print(submission.head())
print("...")
print(submission.tail())

Creating submission...
Shape: (20100, 4)
Expected: (20100, 4)

Sample:
      id           x           y        deg
0  001_0   s0.000000   s0.000000  s0.000000
1  002_0  s-0.459000  s-0.400000  s0.000000
2  002_1   s0.459000  s-0.400000  s0.000000
3  003_0  s-0.440450  s-0.993190  s0.000000
4  003_1   s0.466607  s-1.024182  s0.000000
...
            id           x          y        deg
20095  200_195  s-4.428233  s6.060686  s0.000000
20096  200_196  s-3.482147  s6.035909  s0.000000
20097  200_197  s-2.543919  s6.013868  s0.000000
20098  200_198  s-1.612924  s5.996910  s0.000000
20099  200_199  s-0.687229  s5.987317  s0.000000


In [ ]:
# ============================================================
# SAVE
# ============================================================

# Final check
all_valid = all(
    not check_any_collision(solutions[n], EPSILON) 
    for n in range(1, MAX_N + 1)
)

if all_valid:
    submission.to_csv('submission.csv', index=False)
    print("✅ submission.csv saved!")
    print(f"Size: {os.path.getsize('submission.csv') / 1024:.1f} KB")
else:
    print("❌ Cannot save - validation failed")

✅ submission.csv saved!
Size: 750.2 KB


In [ ]:
# ============================================================
# SUMMARY
# ============================================================

print("\n" + "="*60)
print("🎄 SUMMARY")
print("="*60)
print(f"Puzzles: {len(solutions)}/{MAX_N}")
print(f"Valid: {all_valid}")
print(f"Time: {total_time:.1f}s")
print(f"Score: {score:.4f}")
print("="*60)


🎄 SUMMARY
Puzzles: 200/200
Valid: True
Time: 2696.8s
Score: 259.7732
